In [9]:
from io import StringIO

import matplotlib.pyplot as plt
from scipy.io import arff
import seaborn as sns
from loguru import logger
import yaml

from datetime import datetime
import polars as pl
import pandas as pd
import numpy as np
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

# MODEL
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import (
    LabelEncoder, 
    StandardScaler,
    label_binarize
)
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    precision_recall_fscore_support,
    balanced_accuracy_score,
    roc_auc_score,
    roc_curve
)

# Save variables for model
import joblib

# PERSONAL FUNCTIONS
from utils import *
from models.main import *
# from functions.windows import create_feature_windows # creación de ventanas e ingenieria de características

In [55]:
with open(r'F:\UPC\Tesis\HARbit-Model\src\config\activities.yaml', 'r') as file:
    config = yaml.safe_load(file)['config']

In [56]:
activities_ = config['labels']
cluster_ = config['clusters']

In [31]:
def create_raw_windows_250_timesteps_robust(df, window_seconds=5, overlap_percent=50, 
                                           sampling_rate=20, target_timesteps=250,
                                           min_data_threshold=0.5, max_gap_seconds=1.0):
    """
    Versión ROBUSTA: Crea ventanas basadas en TIEMPO REAL con validación mejorada
    
    Args:
        df: DataFrame con datos de sensores (Polars o Pandas)
        window_seconds: Duración de la ventana en segundos (default: 5)
        overlap_percent: Porcentaje de solapamiento (default: 50)
        sampling_rate: Frecuencia de muestreo en Hz (default: 20)
        target_timesteps: Número objetivo de timesteps por ventana (default: 250)
        min_data_threshold: Umbral mínimo de datos válidos (0.5 = 50%)
        max_gap_seconds: Máximo gap permitido en segundos (1.0s)
        
    Returns:
        X: Array con forma (n_windows, 250, 3) - datos de ventanas
        y: Array con etiquetas de actividad
        subjects: Array con IDs de usuario
        metadata: DataFrame con información de las ventanas
    """
    
    print(f"🔧 Configuración de ventanas RAW ROBUSTA:")
    print(f"  Duración: {window_seconds}s")
    print(f"  Timesteps objetivo: {target_timesteps}")
    print(f"  Frecuencia de muestreo: {sampling_rate}Hz")
    print(f"  Solapamiento: {overlap_percent}%")
    print(f"  Umbral mínimo de datos: {min_data_threshold*100:.1f}%")
    print(f"  Máximo gap permitido: {max_gap_seconds}s")
    
    # Convertir a pandas si es necesario
    if hasattr(df, 'to_pandas'):
        df_pd = df.to_pandas()
    else:
        df_pd = df.copy()
    
    # Asegurar que Timestamp es datetime
    if df_pd['Timestamp'].dtype == 'object':
        df_pd['Timestamp'] = pd.to_datetime(df_pd['Timestamp'])
    elif df_pd['Timestamp'].dtype == 'int64':
        df_pd['Timestamp'] = pd.to_datetime(df_pd['Timestamp'])
    
    # Calcular parámetros de tiempo
    window_duration_ns = int(window_seconds * 1e9)
    step_duration_ns = int(window_duration_ns * (100 - overlap_percent) / 100)
    
    print(f"  Duración de ventana: {window_seconds}s")
    print(f"  Paso entre ventanas: {step_duration_ns / 1e9:.2f}s")
    
    # Listas para almacenar resultados
    X_windows = []
    y_labels = []
    subjects_list = []
    metadata_list = []
    
    total_windows_attempted = 0
    total_windows_created = 0
    
    # Procesar por usuario y actividad
    for (user_id, activity), group in df_pd.groupby(['Subject-id', 'Activity Label']):
        
        # Ordenar por timestamp y limpiar datos
        group = group.sort_values('Timestamp').reset_index(drop=True)
        group = group.dropna(subset=['X', 'Y', 'Z', 'Timestamp'])
        
        if len(group) < window_seconds * sampling_rate:
            print(f"⚠️ Usuario {user_id}, Actividad {activity}: Muy pocos datos ({len(group)} muestras)")
            continue
        
        # Convertir timestamps a nanosegundos
        if group['Timestamp'].dtype.name.startswith('datetime'):
            timestamps_ns = group['Timestamp'].astype('int64')
        else:
            timestamps_ns = group['Timestamp'].values
        
        print(f"👤 Usuario {user_id}, Actividad {activity}: {len(group)} muestras")
        
        # Obtener rango temporal
        start_time_ns = timestamps_ns.min()
        end_time_ns = timestamps_ns.max()
        total_duration_s = (end_time_ns - start_time_ns) / 1e9
        
        print(f"   Duración total: {total_duration_s:.1f}s")
        
        # Detectar y reportar gaps grandes
        time_diffs = np.diff(timestamps_ns) / 1e9  # Convertir a segundos
        large_gaps = time_diffs > max_gap_seconds
        if np.any(large_gaps):
            n_gaps = np.sum(large_gaps)
            max_gap = np.max(time_diffs)
            print(f"   ⚠️ Detectados {n_gaps} gaps > {max_gap_seconds}s (máximo: {max_gap:.1f}s)")
        
        # Crear ventanas deslizantes
        window_count = 0
        current_start_ns = start_time_ns
        
        while current_start_ns + window_duration_ns <= end_time_ns:
            total_windows_attempted += 1
            current_end_ns = current_start_ns + window_duration_ns
            
            # Filtrar datos en esta ventana temporal
            window_mask = (
                (timestamps_ns >= current_start_ns) & 
                (timestamps_ns < current_end_ns)
            )
            window_data_df = group[window_mask]
            
            # Validación de ventana
            is_valid, validation_info = validate_window_data(
                window_data_df, 
                window_seconds, 
                sampling_rate, 
                min_data_threshold,
                max_gap_seconds
            )
            
            if is_valid:
                # Extraer datos de sensores
                sensor_data = window_data_df[['X', 'Y', 'Z']].values
                window_timestamps = window_data_df['Timestamp'].values
                
                try:
                    # Redimensionar/interpolar a target_timesteps
                    resampled_window = resample_window_robust(
                        sensor_data, window_timestamps, target_timesteps, window_seconds
                    )
                    
                    # Verificar calidad final
                    if is_window_quality_good(resampled_window):
                        # Guardar datos
                        X_windows.append(resampled_window)
                        y_labels.append(activity)
                        subjects_list.append(user_id)
                        
                        # Metadata extendida
                        metadata_list.append({
                            'Subject-id': user_id,
                            'Activity Label': activity,
                            'window_start': pd.to_datetime(current_start_ns),
                            'window_end': pd.to_datetime(current_end_ns),
                            'original_samples': len(window_data_df),
                            'resampled_timesteps': target_timesteps,
                            'window_idx': window_count,
                            'actual_duration_s': window_seconds,
                            'data_coverage': validation_info['data_coverage'],
                            'max_gap_s': validation_info['max_gap'],
                            'sampling_rate_actual': validation_info['actual_rate']
                        })
                        
                        window_count += 1
                        total_windows_created += 1
                    else:
                        print(f"   ❌ Ventana {window_count}: Calidad de datos insuficiente después de interpolación")
                
                except Exception as e:
                    print(f"   ❌ Ventana {window_count}: Error en interpolación - {str(e)}")
            
            else:
                # No mostrar warning para cada ventana inválida, solo resumen
                pass
            
            # Mover al siguiente inicio de ventana
            current_start_ns += step_duration_ns
        
        print(f"  ✅ Creadas {window_count} ventanas válidas")
    
    # Resumen final
    print(f"\n📊 RESUMEN DE VALIDACIÓN:")
    print(f"  Ventanas intentadas: {total_windows_attempted}")
    print(f"  Ventanas creadas: {total_windows_created}")
    print(f"  Tasa de éxito: {(total_windows_created/total_windows_attempted)*100:.1f}%")
    
    # Convertir a arrays numpy
    if len(X_windows) > 0:
        X = np.array(X_windows)
        y = np.array(y_labels)
        subjects = np.array(subjects_list)
        metadata_df = pd.DataFrame(metadata_list)
        
        print(f"\n📊 RESULTADO FINAL (ROBUSTO):")
        print(f"  Forma de X: {X.shape}")
        print(f"  Forma de y: {y.shape}")
        print(f"  Total ventanas: {len(X)}")
        print(f"  Usuarios únicos: {len(np.unique(subjects))}")
        print(f"  Actividades únicas: {sorted(np.unique(y))}")
        
        return X, y, subjects, metadata_df
    
    else:
        print("❌ No se crearon ventanas válidas")
        return None, None, None, None


def validate_window_data(window_data_df, window_seconds, sampling_rate, 
                        min_data_threshold, max_gap_seconds):
    """
    Valida si una ventana de datos es aceptable
    
    Returns:
        bool: True si la ventana es válida
        dict: Información de validación
    """
    if len(window_data_df) == 0:
        return False, {'reason': 'empty', 'data_coverage': 0, 'max_gap': float('inf'), 'actual_rate': 0}
    
    # Calcular cobertura de datos esperada
    expected_samples = window_seconds * sampling_rate
    actual_samples = len(window_data_df)
    data_coverage = actual_samples / expected_samples
    
    # Si hay muy pocos datos
    if data_coverage < min_data_threshold:
        return False, {
            'reason': 'insufficient_data', 
            'data_coverage': data_coverage,
            'max_gap': float('inf'),
            'actual_rate': 0
        }
    
    # Calcular gaps en los datos
    if len(window_data_df) > 1:
        timestamps = pd.to_datetime(window_data_df['Timestamp'])
        time_diffs = timestamps.diff().dt.total_seconds().fillna(0)
        max_gap = time_diffs.max()
        actual_rate = len(window_data_df) / (timestamps.max() - timestamps.min()).total_seconds()
    else:
        max_gap = 0
        actual_rate = sampling_rate
    
    # Si hay gaps muy grandes
    if max_gap > max_gap_seconds:
        return False, {
            'reason': 'large_gap', 
            'data_coverage': data_coverage,
            'max_gap': max_gap,
            'actual_rate': actual_rate
        }
    
    # Verificar que no hay valores NaN o infinitos en los sensores
    sensor_data = window_data_df[['X', 'Y', 'Z']].values
    if np.any(np.isnan(sensor_data)) or np.any(np.isinf(sensor_data)):
        return False, {
            'reason': 'invalid_values',
            'data_coverage': data_coverage,
            'max_gap': max_gap,
            'actual_rate': actual_rate
        }
    
    return True, {
        'reason': 'valid',
        'data_coverage': data_coverage,
        'max_gap': max_gap,
        'actual_rate': actual_rate
    }


def resample_window_robust(sensor_data, timestamps, target_timesteps, window_seconds):
    """
    Versión robusta de remuestreo con múltiples estrategias
    """
    from scipy.interpolate import interp1d
    from scipy import signal
    
    if len(sensor_data) == 0:
        return np.zeros((target_timesteps, 3))
    
    original_timesteps = len(sensor_data)
    
    if original_timesteps == target_timesteps:
        return sensor_data.copy()
    
    if original_timesteps == 1:
        return np.tile(sensor_data[0], (target_timesteps, 1))
    
    try:
        # Estrategia 1: Interpolación temporal precisa
        if hasattr(timestamps[0], 'timestamp'):
            time_seconds = np.array([t.timestamp() for t in timestamps])
        elif isinstance(timestamps[0], pd.Timestamp):
            time_seconds = np.array([t.timestamp() for t in timestamps])
        else:
            time_seconds = timestamps.astype('int64') / 1e9
        
        # Normalizar tiempos
        time_min = time_seconds.min()
        time_max = time_seconds.max()
        
        if time_max > time_min:
            relative_times = (time_seconds - time_min) / (time_max - time_min)
        else:
            relative_times = np.linspace(0, 1, len(time_seconds))
        
        # Crear tiempos objetivo uniformes
        target_times = np.linspace(0, 1, target_timesteps)
        
        # Interpolar cada eje
        resampled_data = np.zeros((target_timesteps, 3))
        
        for axis in range(3):
            try:
                # Estrategia de interpolación según la cantidad de datos
                if original_timesteps >= target_timesteps:
                    # Downsample: usar signal.resample para preservar características
                    resampled_axis = signal.resample(sensor_data[:, axis], target_timesteps)
                else:
                    # Upsample: usar interpolación
                    if len(np.unique(relative_times)) > 1:
                        interpolator = interp1d(
                            relative_times, 
                            sensor_data[:, axis],
                            kind='cubic' if original_timesteps >= 4 else 'linear',
                            bounds_error=False,
                            fill_value='extrapolate'
                        )
                        resampled_axis = interpolator(target_times)
                    else:
                        resampled_axis = np.full(target_timesteps, sensor_data[0, axis])
                
                resampled_data[:, axis] = resampled_axis
                
            except Exception as e:
                # Fallback: interpolación lineal simple
                resampled_data[:, axis] = np.interp(
                    target_times, relative_times, sensor_data[:, axis]
                )
        
        return resampled_data
    
    except Exception as e:
        print(f"Error en remuestreo robusto: {str(e)}")
        # Último fallback: replicar la primera muestra
        return np.tile(sensor_data[0], (target_timesteps, 1))


def is_window_quality_good(resampled_window, max_std_threshold=50.0):
    """
    Verifica la calidad final de una ventana remuestreada
    """
    # Verificar NaN o infinitos
    if np.any(np.isnan(resampled_window)) or np.any(np.isinf(resampled_window)):
        return False
    
    # Verificar valores extremos (posibles errores de interpolación)
    if np.any(np.abs(resampled_window) > 1000):  # Ajustar según tus datos
        return False
    
    # Verificar varianza (datos demasiado planos pueden indicar error)
    for axis in range(resampled_window.shape[1]):
        std_axis = np.std(resampled_window[:, axis])
        if std_axis > max_std_threshold:  # Varianza excesiva
            return False
        if std_axis < 0.001:  # Datos demasiado planos
            return False
    
    return True

In [ ]:
# Análisis rápido de tiempos por usuario y actividad
# time_analysis = df_accel.to_pandas().groupby(['Subject-id', 'Activity Label'])['Timestamp'].agg(['min', 'max']).reset_index()
# time_analysis['duration_seconds'] = (pd.to_datetime(time_analysis['max']) - pd.to_datetime(time_analysis['min'])).dt.total_seconds()
# time_analysis['duration_hours'] = time_analysis['duration_seconds'] / 3600

# # Mostrar casos extremos
# print("🔍 DURACIONES POR USUARIO Y ACTIVIDAD:")
# time_analysis.sort_values('duration_seconds', ascending=False)

🔍 DURACIONES POR USUARIO Y ACTIVIDAD:


In [63]:
def loso_cross_validation_raw_windows_robust(df_accel, df_gyro=None, model_architecture_func=None,
                                           window_seconds=5, overlap_percent=50, 
                                           sampling_rate=20, target_timesteps=250,
                                           epochs=30, batch_size=32, verbose=1, 
                                           save_results=True, results_path="loso_raw_windows"):
    """LOSO con validación robusta de ventanas"""
    
    print("🚀 INICIANDO LOSO CROSS-VALIDATION - VENTANAS RAW ROBUSTAS")
    print("=" * 70)
    
    # Usar la función robusta para crear ventanas
    X_all, y_all, subjects_all, metadata_all = create_raw_windows_250_timesteps_robust(
        df=df_accel,
        window_seconds=window_seconds,
        overlap_percent=overlap_percent,
        sampling_rate=sampling_rate,
        target_timesteps=target_timesteps,
        min_data_threshold=0.8,  # 80% mínimo de datos
        max_gap_seconds=1.0      # Máximo 1 segundo de gap
    )

    y_all_mapped = []
    for label in y_all:
        # Primero mapear con activities_
        mapped_activity = activities_[label]
        
        # Luego buscar en qué cluster está
        final_cluster = mapped_activity  # Por defecto, mantener la actividad
        for cluster_name, activities_in_cluster in cluster_.items():
            if mapped_activity in activities_in_cluster:
                final_cluster = cluster_name
                break
        
        y_all_mapped.append(final_cluster)
    
    y_all = np.array(y_all_mapped)
    print("Actividades únicas después del mapeo:", np.unique(y_all))
    
    if X_all is None:
        print("❌ No se pudieron crear ventanas válidas")
        return None
    
    # Obtener usuarios únicos
    users = np.unique(subjects_all)
    n_users = len(users)
    
    print(f"👥 Total de usuarios: {n_users}")
    print(f"📊 Usuarios: {users}")
    print(f"📊 Forma de datos: {X_all.shape}")
    
    # Preparar label encoder global
    from sklearn.preprocessing import LabelEncoder
    global_label_encoder = LabelEncoder()
    y_all_encoded = global_label_encoder.fit_transform(y_all)
    
    # Almacenar resultados
    loso_results = {
        'user': [],
        'accuracy': [],
        'precision_macro': [],
        'recall_macro': [],
        'f1_macro': [],
        'confusion_matrix': [],
        'classification_report': [],
        'train_windows': [],
        'test_windows': [],
        'y_true': [],
        'y_pred': []
    }
    
    all_y_true = []
    all_y_pred = []
    
    # Iterar sobre cada usuario (LOSO)
    for i, test_user in enumerate(users):
        print(f"\n🔄 FOLD {i+1}/{n_users}: Usuario de test = {test_user}")
        print("-" * 50)
        
        # División de datos: un usuario para test, resto para train
        train_mask = subjects_all != test_user
        test_mask = subjects_all == test_user
        
        X_train = X_all[train_mask]
        y_train = y_all_encoded[train_mask]
        X_test = X_all[test_mask]
        y_test = y_all_encoded[test_mask]
        
        print(f"📊 Train: {len(X_train)} ventanas de {len(np.unique(subjects_all[train_mask]))} usuarios")
        print(f"📊 Test: {len(X_test)} ventanas del usuario {test_user}")
        
        # Verificar que hay suficientes datos
        if len(X_train) < 50 or len(X_test) < 10:
            print(f"⚠️ Datos insuficientes para usuario {test_user}, saltando...")
            continue
        
        # Verificar clases en común
        train_classes = set(np.unique(y_train))
        test_classes = set(np.unique(y_test))
        common_classes = train_classes.intersection(test_classes)
        
        if len(common_classes) < 2:
            print(f"⚠️ Muy pocas clases comunes para usuario {test_user}, saltando...")
            continue
        
        print(f"🎯 Clases comunes: {len(common_classes)}")
        
        try:
            # Crear y compilar modelo
            input_shape = (target_timesteps, X_train.shape[2])
            num_classes = len(global_label_encoder.classes_)
            
            if model_architecture_func is None:
                model = create_cnn_lstm_model(input_shape=input_shape, num_classes=num_classes)
            else:
                model = model_architecture_func(input_shape=input_shape, num_classes=num_classes)
            
            # Entrenar modelo
            print(f"🏋️ Entrenando modelo...")
            history = model.fit(
                X_train, y_train,
                epochs=epochs,
                batch_size=batch_size,
                callbacks=callbacks,
                verbose=0 if verbose == 0 else 1,
                validation_split=0.1
            )
            
            # Evaluar modelo
            print(f"🔍 Evaluando modelo...")
            y_pred = model.predict(X_test, verbose=0)
            y_pred_classes = np.argmax(y_pred, axis=1)
            
            # Calcular métricas
            from sklearn.metrics import accuracy_score, precision_recall_fscore_support
            from sklearn.metrics import classification_report, confusion_matrix
            
            accuracy = accuracy_score(y_test, y_pred_classes)
            precision, recall, f1, _ = precision_recall_fscore_support(
                y_test, y_pred_classes, average='macro', zero_division=0
            )
            
            # Reporte de clasificación
            report = classification_report(
                y_test, y_pred_classes,
                target_names=global_label_encoder.classes_,
                output_dict=True,
                zero_division=0
            )
            
            # Matriz de confusión
            cm = confusion_matrix(y_test, y_pred_classes)
            
            # Guardar resultados
            loso_results['user'].append(test_user)
            loso_results['accuracy'].append(accuracy)
            loso_results['precision_macro'].append(precision)
            loso_results['recall_macro'].append(recall)
            loso_results['f1_macro'].append(f1)
            loso_results['confusion_matrix'].append(cm)
            loso_results['classification_report'].append(report)
            loso_results['train_windows'].append(len(X_train))
            loso_results['test_windows'].append(len(X_test))
            loso_results['y_true'].append(y_test)
            loso_results['y_pred'].append(y_pred_classes)
            
            # Acumular para métricas globales
            all_y_true.extend(y_test)
            all_y_pred.extend(y_pred_classes)
            
            print(f"✅ Usuario {test_user}: Accuracy = {accuracy:.4f}")
            
        except Exception as e:
            print(f"❌ Error con usuario {test_user}: {str(e)}")
            continue
    
    # Calcular métricas agregadas
    print(f"\n📊 RESULTADOS LOSO RAW WINDOWS")
    print("=" * 70)
    
    if len(loso_results['accuracy']) > 0:
        mean_accuracy = np.mean(loso_results['accuracy'])
        std_accuracy = np.std(loso_results['accuracy'])
        mean_precision = np.mean(loso_results['precision_macro'])
        mean_recall = np.mean(loso_results['recall_macro'])
        mean_f1 = np.mean(loso_results['f1_macro'])
        
        print(f"🎯 Accuracy promedio: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
        print(f"🎯 Precision promedio: {mean_precision:.4f}")
        print(f"🎯 Recall promedio: {mean_recall:.4f}")
        print(f"🎯 F1-Score promedio: {mean_f1:.4f}")
        print(f"📊 Usuarios evaluados: {len(loso_results['accuracy'])}/{n_users}")
        
        # Guardar resultados
        if save_results:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            
            results_df = pd.DataFrame({
                'user': loso_results['user'],
                'accuracy': loso_results['accuracy'],
                'precision_macro': loso_results['precision_macro'],
                'recall_macro': loso_results['recall_macro'],
                'f1_macro': loso_results['f1_macro'],
                'train_windows': loso_results['train_windows'],
                'test_windows': loso_results['test_windows']
            })
            
            results_df.to_csv(f"{results_path}_summary_{timestamp}.csv", index=False)
            joblib.dump(loso_results, f"{results_path}_complete_{timestamp}.joblib")
            
            print(f"💾 Resultados guardados: {results_path}_*_{timestamp}.*")
        
        return {
            'summary': results_df,
            'model': model,
            'mean_accuracy': mean_accuracy,
            'std_accuracy': std_accuracy,
            'mean_precision': mean_precision,
            'mean_recall': mean_recall,
            'mean_f1': mean_f1,
            'detailed_results': loso_results,
            'global_label_encoder': global_label_encoder
        }
    
    else:
        print("❌ No se pudieron evaluar usuarios")
        return None

In [58]:
# Cargar datos
path_base = r"F:\UPC\Tesis\HARbit-Model\src\data\wisdm-dataset\raw\watch"
sensor_data = load_sensors_separately(path_base)
df_gyro = sensor_data['gyro']
# df_gyro = df_gyro.rename({'X': 'X_gyro', 'Y': 'Y_gyro', 'Z': 'Z_gyro'})

df_accel = sensor_data['accel']
# df_accel = df_accel.rename({'X': 'X_accel', 'Y': 'Y_accel', 'Z': 'Z_accel'})

print(f"Giroscopio: {len(df_gyro)} muestras")
print(f"Acelerómetro: {len(df_accel)} muestras")

2025-09-23 22:24:11.834 | INFO     | utils.data_loading:load_sensors_separately:64 - Cargando datos de gyro...


2025-09-23 22:24:23.267 | INFO     | utils.data_loading:load_sensors_separately:64 - Cargando datos de accel...


Giroscopio: 3440341 muestras
Acelerómetro: 3777045 muestras


In [59]:

# df_gyro = df_gyro.to_pandas()
# df_accel = df_accel.to_pandas()

# df_gyro['Activity Label']   = df_gyro['Activity Label'].apply(lambda x: activities_[x])
# df_accel['Activity Label']  = df_accel['Activity Label'].apply(lambda x: activities_[x])

# for activity in cluster_.keys():
#     for act in cluster_[activity]:
#         df_gyro.loc[df_gyro['Activity Label'] == act, 'Activity Label'] = activity
#         df_accel.loc[df_accel['Activity Label'] == act, 'Activity Label'] = activity

In [37]:
def compare_data_before_after_processing(df_original, X_processed, y_processed, subjects_processed, 
                                        metadata_df, window_seconds=5, target_timesteps=250):
    """
    Compara los datos originales con los procesados para validar el tratamiento
    
    Args:
        df_original: DataFrame original con datos de sensores
        X_processed: Array procesado con ventanas (n_windows, 250, 3)
        y_processed: Array con etiquetas procesadas
        subjects_processed: Array con IDs de usuario procesados
        metadata_df: DataFrame con metadata de las ventanas
        window_seconds: Duración de ventana en segundos
        target_timesteps: Timesteps objetivo por ventana
        
    Returns:
        dict: Reporte completo de comparación
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    print("🔍 COMPARACIÓN DATOS ORIGINALES vs PROCESADOS")
    print("=" * 60)
    
    # ==================== ANÁLISIS GENERAL ====================
    print("\n📊 ESTADÍSTICAS GENERALES:")
    print("-" * 40)
    
    # Datos originales
    original_samples = len(df_original)
    original_users = df_original['Subject-id'].nunique()
    original_activities = df_original['Activity Label'].nunique()
    original_duration = (pd.to_datetime(df_original['Timestamp']).max() - 
                        pd.to_datetime(df_original['Timestamp']).min()).total_seconds()
    
    # Datos procesados
    processed_windows = len(X_processed) if X_processed is not None else 0
    processed_users = len(np.unique(subjects_processed)) if subjects_processed is not None else 0
    processed_activities = len(np.unique(y_processed)) if y_processed is not None else 0
    processed_duration = processed_windows * window_seconds if processed_windows > 0 else 0
    
    print(f"📈 ORIGINALES:")
    print(f"  Muestras totales: {original_samples:,}")
    print(f"  Usuarios únicos: {original_users}")
    print(f"  Actividades únicas: {original_activities}")
    print(f"  Duración total: {original_duration/3600:.1f} horas")
    
    print(f"\n📊 PROCESADOS:")
    print(f"  Ventanas creadas: {processed_windows:,}")
    print(f"  Usuarios conservados: {processed_users}")
    print(f"  Actividades conservadas: {processed_activities}")
    print(f"  Duración equivalente: {processed_duration/3600:.1f} horas")
    
    # Eficiencia de conversión
    if original_samples > 0:
        conversion_rate = (processed_windows * target_timesteps) / original_samples * 100
        print(f"\n🎯 EFICIENCIA DE CONVERSIÓN:")
        print(f"  Tasa de aprovechamiento: {conversion_rate:.1f}%")
        print(f"  Reducción de datos: {(1 - conversion_rate/100)*100:.1f}%")
    
    # ==================== ANÁLISIS POR USUARIO ====================
    print(f"\n👥 ANÁLISIS POR USUARIO:")
    print("-" * 40)
    
    user_comparison = []
    
    if X_processed is not None:
        for user_id in np.unique(subjects_processed):
            # Datos originales del usuario
            original_user_data = df_original[df_original['Subject-id'] == user_id]
            original_user_samples = len(original_user_data)
            original_user_activities = original_user_data['Activity Label'].nunique()
            
            # Datos procesados del usuario
            processed_user_windows = np.sum(subjects_processed == user_id)
            processed_user_activities = len(np.unique(y_processed[subjects_processed == user_id]))
            
            user_comparison.append({
                'user_id': user_id,
                'original_samples': original_user_samples,
                'original_activities': original_user_activities,
                'processed_windows': processed_user_windows,
                'processed_activities': processed_user_activities,
                'conversion_rate': (processed_user_windows * target_timesteps) / original_user_samples * 100 if original_user_samples > 0 else 0
            })
            
            print(f"Usuario {user_id}: {original_user_samples} → {processed_user_windows} ventanas ({processed_user_windows * target_timesteps} puntos)")
    
    # ==================== ANÁLISIS POR ACTIVIDAD ====================
    print(f"\n🏃 ANÁLISIS POR ACTIVIDAD:")
    print("-" * 40)
    
    activity_comparison = []
    
    if X_processed is not None:
        for activity in np.unique(y_processed):
            # Datos originales de la actividad
            original_activity_data = df_original[df_original['Activity Label'] == activity]
            original_activity_samples = len(original_activity_data)
            original_activity_users = original_activity_data['Subject-id'].nunique()
            
            # Datos procesados de la actividad
            processed_activity_windows = np.sum(y_processed == activity)
            processed_activity_users = len(np.unique(subjects_processed[y_processed == activity]))
            
            activity_comparison.append({
                'activity': activity,
                'original_samples': original_activity_samples,
                'original_users': original_activity_users,
                'processed_windows': processed_activity_windows,
                'processed_users': processed_activity_users,
                'conversion_rate': (processed_activity_windows * target_timesteps) / original_activity_samples * 100 if original_activity_samples > 0 else 0
            })
            
            print(f"{activity}: {original_activity_samples} → {processed_activity_windows} ventanas")
    
    # ==================== ANÁLISIS DE CALIDAD DE DATOS ====================
    print(f"\n🔬 ANÁLISIS DE CALIDAD:")
    print("-" * 40)
    
    if X_processed is not None:
        # Estadísticas de los datos procesados
        print(f"📊 Estadísticas de ventanas procesadas:")
        print(f"  Forma de X: {X_processed.shape}")
        print(f"  Rango de valores:")
        print(f"    Min: {X_processed.min():.3f}")
        print(f"    Max: {X_processed.max():.3f}")
        print(f"    Media: {X_processed.mean():.3f}")
        print(f"    Std: {X_processed.std():.3f}")
        
        # Verificar datos anómalos
        nan_count = np.isnan(X_processed).sum()
        inf_count = np.isinf(X_processed).sum()
        extreme_count = np.sum(np.abs(X_processed) > 1000)
        
        print(f"\n🚨 Verificación de anomalías:")
        print(f"  Valores NaN: {nan_count}")
        print(f"  Valores infinitos: {inf_count}")
        print(f"  Valores extremos (>1000): {extreme_count}")
        
        if nan_count == 0 and inf_count == 0 and extreme_count == 0:
            print("  ✅ Sin anomalías detectadas")
        else:
            print("  ❌ Se encontraron anomalías")
    
    # ==================== ANÁLISIS TEMPORAL ====================
    if metadata_df is not None and len(metadata_df) > 0:
        print(f"\n⏰ ANÁLISIS TEMPORAL:")
        print("-" * 40)
        
        # Cobertura de datos por ventana
        coverage_stats = metadata_df['data_coverage'].describe()
        print(f"📊 Cobertura de datos por ventana:")
        print(f"  Promedio: {coverage_stats['mean']:.3f}")
        print(f"  Mínimo: {coverage_stats['min']:.3f}")
        print(f"  Máximo: {coverage_stats['max']:.3f}")
        
        # Gaps por ventana
        gap_stats = metadata_df['max_gap_s'].describe()
        print(f"\n🕳️ Gaps por ventana:")
        print(f"  Gap promedio: {gap_stats['mean']:.3f}s")
        print(f"  Gap máximo: {gap_stats['max']:.3f}s")
        
        # Frecuencia de muestreo real
        rate_stats = metadata_df['sampling_rate_actual'].describe()
        print(f"\n📡 Frecuencia de muestreo real:")
        print(f"  Promedio: {rate_stats['mean']:.1f}Hz")
        print(f"  Rango: {rate_stats['min']:.1f} - {rate_stats['max']:.1f}Hz")
    
    # ==================== VISUALIZACIONES ====================
    if X_processed is not None:
        create_comparison_visualizations(
            df_original, X_processed, y_processed, subjects_processed, 
            user_comparison, activity_comparison, metadata_df
        )
    
    # ==================== REPORTE FINAL ====================
    report = {
        'original_stats': {
            'samples': original_samples,
            'users': original_users,
            'activities': original_activities,
            'duration_hours': original_duration/3600
        },
        'processed_stats': {
            'windows': processed_windows,
            'users': processed_users,
            'activities': processed_activities,
            'duration_hours': processed_duration/3600
        },
        'conversion_rate': conversion_rate if 'conversion_rate' in locals() else 0,
        'user_comparison': user_comparison,
        'activity_comparison': activity_comparison,
        'quality_check': {
            'nan_count': nan_count if 'nan_count' in locals() else 0,
            'inf_count': inf_count if 'inf_count' in locals() else 0,
            'extreme_count': extreme_count if 'extreme_count' in locals() else 0
        }
    }
    
    return report


def create_comparison_visualizations(df_original, X_processed, y_processed, subjects_processed, 
                                   user_comparison, activity_comparison, metadata_df):
    """
    Crea visualizaciones para comparar datos originales vs procesados
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Comparación: Datos Originales vs Procesados', fontsize=16, fontweight='bold')
    
    # 1. Distribución por usuarios
    if user_comparison:
        user_df = pd.DataFrame(user_comparison)
        
        axes[0,0].bar(range(len(user_df)), user_df['original_samples'], alpha=0.7, label='Original', color='lightblue')
        axes[0,0].bar(range(len(user_df)), user_df['processed_windows'] * 250, alpha=0.7, label='Procesado', color='orange')
        axes[0,0].set_title('Muestras por Usuario')
        axes[0,0].set_xlabel('Usuario')
        axes[0,0].set_ylabel('Número de Muestras')
        axes[0,0].legend()
        axes[0,0].set_xticks(range(len(user_df)))
        axes[0,0].set_xticklabels([f"U{int(u)}" for u in user_df['user_id']], rotation=45)
    
    # 2. Distribución por actividades
    if activity_comparison:
        activity_df = pd.DataFrame(activity_comparison)
        
        x_pos = range(len(activity_df))
        width = 0.35
        
        axes[0,1].bar([x - width/2 for x in x_pos], activity_df['original_samples'], 
                      width, label='Original', alpha=0.7, color='lightblue')
        axes[0,1].bar([x + width/2 for x in x_pos], activity_df['processed_windows'] * 250, 
                      width, label='Procesado', alpha=0.7, color='orange')
        axes[0,1].set_title('Muestras por Actividad')
        axes[0,1].set_xlabel('Actividad')
        axes[0,1].set_ylabel('Número de Muestras')
        axes[0,1].legend()
        axes[0,1].set_xticks(x_pos)
        axes[0,1].set_xticklabels(activity_df['activity'], rotation=45)
    
    # 3. Tasas de conversión por usuario
    if user_comparison:
        axes[0,2].bar(range(len(user_df)), user_df['conversion_rate'], alpha=0.7, color='green')
        axes[0,2].set_title('Tasa de Conversión por Usuario')
        axes[0,2].set_xlabel('Usuario')
        axes[0,2].set_ylabel('Tasa de Conversión (%)')
        axes[0,2].set_xticks(range(len(user_df)))
        axes[0,2].set_xticklabels([f"U{int(u)}" for u in user_df['user_id']], rotation=45)
    
    # 4. Distribución de valores originales vs procesados
    original_sample = df_original[['X', 'Y', 'Z']].sample(10000) if len(df_original) > 10000 else df_original[['X', 'Y', 'Z']]
    processed_sample = X_processed.reshape(-1, X_processed.shape[-1])
    processed_sample = processed_sample[::max(1, len(processed_sample)//10000)]  # Muestra de 10k
    
    axes[1,0].hist(original_sample.values.flatten(), bins=50, alpha=0.5, label='Original', density=True)
    axes[1,0].hist(processed_sample.flatten(), bins=50, alpha=0.5, label='Procesado', density=True)
    axes[1,0].set_title('Distribución de Valores de Sensores')
    axes[1,0].set_xlabel('Valor del Sensor')
    axes[1,0].set_ylabel('Densidad')
    axes[1,0].legend()
    
    # 5. Cobertura de datos (si hay metadata)
    if metadata_df is not None and len(metadata_df) > 0:
        axes[1,1].hist(metadata_df['data_coverage'], bins=30, alpha=0.7, color='purple')
        axes[1,1].set_title('Distribución de Cobertura de Datos')
        axes[1,1].set_xlabel('Cobertura de Datos')
        axes[1,1].set_ylabel('Frecuencia')
        axes[1,1].axvline(metadata_df['data_coverage'].mean(), color='red', linestyle='--', 
                         label=f'Media: {metadata_df["data_coverage"].mean():.3f}')
        axes[1,1].legend()
    
    # 6. Ejemplo de ventana procesada
    if X_processed is not None and len(X_processed) > 0:
        sample_window = X_processed[0]  # Primera ventana como ejemplo
        axes[1,2].plot(sample_window[:, 0], label='X', alpha=0.7)
        axes[1,2].plot(sample_window[:, 1], label='Y', alpha=0.7)
        axes[1,2].plot(sample_window[:, 2], label='Z', alpha=0.7)
        axes[1,2].set_title('Ejemplo de Ventana Procesada (250 timesteps)')
        axes[1,2].set_xlabel('Timestep')
        axes[1,2].set_ylabel('Valor del Sensor')
        axes[1,2].legend()
        axes[1,2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


def validate_processing_integrity(df_original, X_processed, y_processed, subjects_processed, 
                                window_seconds=5, target_timesteps=250, sampling_rate=20):
    """
    Valida la integridad del procesamiento de datos
    
    Returns:
        dict: Resultado de validación con checks específicos
    """
    print("🔍 VALIDACIÓN DE INTEGRIDAD DEL PROCESAMIENTO")
    print("=" * 50)
    
    validation_results = {
        'shape_check': False,
        'data_range_check': False,
        'temporal_consistency_check': False,
        'user_preservation_check': False,
        'activity_preservation_check': False,
        'overall_valid': False,
        'issues': []
    }
    
    if X_processed is None:
        validation_results['issues'].append("X_processed es None")
        return validation_results
    
    # 1. Verificar forma de datos
    expected_shape = (None, target_timesteps, 3)  # (n_windows, 250, 3)
    actual_shape = X_processed.shape
    
    if len(actual_shape) == 3 and actual_shape[1] == target_timesteps and actual_shape[2] == 3:
        validation_results['shape_check'] = True
        print("✅ Forma de datos correcta:", actual_shape)
    else:
        validation_results['issues'].append(f"Forma incorrecta: {actual_shape}, esperada: (n_windows, {target_timesteps}, 3)")
        print("❌ Forma de datos incorrecta:", actual_shape)
    
    # 2. Verificar rango de datos
    if not np.any(np.isnan(X_processed)) and not np.any(np.isinf(X_processed)):
        data_range = X_processed.max() - X_processed.min()
        if data_range > 0 and X_processed.max() < 1000 and X_processed.min() > -1000:
            validation_results['data_range_check'] = True
            print("✅ Rango de datos válido:", f"{X_processed.min():.3f} a {X_processed.max():.3f}")
        else:
            validation_results['issues'].append(f"Rango de datos sospechoso: {X_processed.min():.3f} a {X_processed.max():.3f}")
            print("❌ Rango de datos sospechoso")
    else:
        validation_results['issues'].append("Datos contienen NaN o infinitos")
        print("❌ Datos contienen NaN o infinitos")
    
    # 3. Verificar consistencia temporal
    expected_total_duration = len(X_processed) * window_seconds
    original_duration = (pd.to_datetime(df_original['Timestamp']).max() - 
                        pd.to_datetime(df_original['Timestamp']).min()).total_seconds()
    
    duration_ratio = expected_total_duration / original_duration
    if 0.1 <= duration_ratio <= 1.5:  # Permitir cierta flexibilidad
        validation_results['temporal_consistency_check'] = True
        print("✅ Consistencia temporal válida:", f"{duration_ratio:.2f} ratio")
    else:
        validation_results['issues'].append(f"Inconsistencia temporal: ratio {duration_ratio:.2f}")
        print("❌ Inconsistencia temporal:", f"ratio {duration_ratio:.2f}")
    
    # 4. Verificar preservación de usuarios
    original_users = set(df_original['Subject-id'].unique())
    processed_users = set(subjects_processed) if subjects_processed is not None else set()
    
    if processed_users.issubset(original_users) and len(processed_users) >= len(original_users) * 0.5:
        validation_results['user_preservation_check'] = True
        print("✅ Preservación de usuarios válida:", f"{len(processed_users)}/{len(original_users)}")
    else:
        validation_results['issues'].append(f"Pérdida excesiva de usuarios: {len(processed_users)}/{len(original_users)}")
        print("❌ Pérdida excesiva de usuarios")
    
    # 5. Verificar preservación de actividades
    original_activities = set(df_original['Activity Label'].unique())
    processed_activities = set(y_processed) if y_processed is not None else set()
    
    if processed_activities.issubset(original_activities) and len(processed_activities) >= len(original_activities) * 0.8:
        validation_results['activity_preservation_check'] = True
        print("✅ Preservación de actividades válida:", f"{len(processed_activities)}/{len(original_activities)}")
    else:
        validation_results['issues'].append(f"Pérdida de actividades: {len(processed_activities)}/{len(original_activities)}")
        print("❌ Pérdida de actividades")
    
    # Verificación general
    all_checks = [
        validation_results['shape_check'],
        validation_results['data_range_check'],
        validation_results['temporal_consistency_check'],
        validation_results['user_preservation_check'],
        validation_results['activity_preservation_check']
    ]
    
    validation_results['overall_valid'] = all(all_checks)
    
    print(f"\n🎯 RESULTADO GENERAL: {'✅ VÁLIDO' if validation_results['overall_valid'] else '❌ INVÁLIDO'}")
    
    if validation_results['issues']:
        print("\n🚨 PROBLEMAS DETECTADOS:")
        for issue in validation_results['issues']:
            print(f"  - {issue}")
    
    return validation_results

In [ ]:
# # Después de procesar tus datos con la función robusta
# X_all, y_all, subjects_all, metadata_all = create_raw_windows_250_timesteps_robust(
#     df=df_accel,
#     window_seconds=5,
#     overlap_percent=50,
#     sampling_rate=20,
#     target_timesteps=250,
#     min_data_threshold=0.6,
#     max_gap_seconds=1.0
# )

# # Realizar comparación completa
# print("🔍 COMPARANDO DATOS ORIGINALES vs PROCESADOS...")
# comparison_report = compare_data_before_after_processing(
#     df_original=df_accel,
#     X_processed=X_all,
#     y_processed=y_all,
#     subjects_processed=subjects_all,
#     metadata_df=metadata_all,
#     window_seconds=5,
#     target_timesteps=250
# )

# # Validar integridad del procesamiento
# print("\n🔍 VALIDANDO INTEGRIDAD...")
# validation_report = validate_processing_integrity(
#     df_original=df_accel,
#     X_processed=X_all,
#     y_processed=y_all,
#     subjects_processed=subjects_all,
#     window_seconds=5,
#     target_timesteps=250,
#     sampling_rate=20
# )

# # Mostrar resumen
# if validation_report['overall_valid']:
#     print("\n🎉 ¡PROCESAMIENTO EXITOSO!")
#     print("Los datos han sido procesados correctamente y están listos para entrenamiento.")
# else:
#     print("\n⚠️ PROCESAMIENTO CO# Después de procesar tus datos con la función robusta")
# X_all, y_all, subjects_all, metadata_all = create_raw_windows_250_timesteps_robust(
#     df=df_accel,
#     window_seconds=5,
#     overlap_percent=50,
#     sampling_rate=20,
#     target_timesteps=250,
#     min_data_threshold=0.6,
#     max_gap_seconds=1.0
# )

# # Realizar comparación completa
# print("🔍 COMPARANDO DATOS ORIGINALES vs PROCESADOS...")
# comparison_report = compare_data_before_after_processing(
#     df_original=df_accel,
#     X_processed=X_all,
#     y_processed=y_all,
#     subjects_processed=subjects_all,
#     metadata_df=metadata_all,
#     window_seconds=5,
#     target_timesteps=250
# )

# # Validar integridad del procesamiento
# print("\n🔍 VALIDANDO INTEGRIDAD...")
# validation_report = validate_processing_integrity(
#     df_original=df_accel,
#     X_processed=X_all,
#     y_processed=y_all,
#     subjects_processed=subjects_all,
#     window_seconds=5,
#     target_timesteps=250,
#     sampling_rate=20
# )

In [ ]:
# Luego ejecuta el LOSO corregido
print("\n🚀 Ejecutando LOSO con ventanas RAW CORREGIDAS...")

# Ejecutar LOSO con la función corregida
loso_results = loso_cross_validation_raw_windows_robust(
    df_accel=df_accel,
    df_gyro=None,
    model_architecture_func=create_cnn_lstm_model,
    window_seconds=5,
    overlap_percent=50,
    sampling_rate=20,
    target_timesteps=250,
    epochs=20,
    batch_size=32,
    verbose=1,
    save_results=True,
    results_path="loso_raw_robust"
)

# Mostrar resultados
if loso_results:
    print(f"✅ LOSO RAW completado!")
    print(f"📊 Accuracy promedio: {loso_results['mean_accuracy']:.4f}")
    print(f"📊 Std Accuracy: {loso_results['std_accuracy']:.4f}")
else:
    print("❌ LOSO falló - revisa los datos")


🚀 Ejecutando LOSO con ventanas RAW CORREGIDAS...
🚀 INICIANDO LOSO CROSS-VALIDATION - VENTANAS RAW ROBUSTAS
🔧 Configuración de ventanas RAW ROBUSTA:
  Duración: 5s
  Timesteps objetivo: 250
  Frecuencia de muestreo: 20Hz
  Solapamiento: 50%
  Umbral mínimo de datos: 80.0%
  Máximo gap permitido: 1.0s
  Duración de ventana: 5s
  Paso entre ventanas: 2.50s
👤 Usuario 1600.0, Actividad A: 3604 muestras
   Duración total: 179.8s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad B: 3605 muestras
   Duración total: 179.9s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad C: 3605 muestras
   Duración total: 179.8s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad D: 3606 muestras
   Duración total: 179.9s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad E: 3605 muestras
   Duración total: 179.9s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad F: 3605 muestras
   Duración total: 179.9s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Activid

In [8]:
print("🚀 Creando ventanas RAW de 250 timesteps...")

# Opción 1: Solo acelerómetro
X_accel_raw, y_accel_raw, subjects_accel, metadata_accel = create_raw_windows_250_timesteps(
    df=df_gyro,  # Tu DataFrame de acelerómetro
    window_seconds=5,
    overlap_percent=50,
    sampling_rate=20,
    target_timesteps=250
)

# Opción 2: Acelerómetro + Giroscopio (RECOMENDADO)
# X_multi_raw, y_multi_raw, subjects_multi, metadata_multi = create_raw_windows_multi_sensor(
#     df_accel=df_accel_inner,
#     df_gyro=df_gyro_inner,
#     window_seconds=5,
#     overlap_percent=50,
#     sampling_rate=20,
#     target_timesteps=250
# )

# Validar ventanas
validate_raw_windows(X_accel_raw, y_accel_raw, subjects_accel, metadata_accel)

# Preparar para entrenamiento
from sklearn.preprocessing import LabelEncoder

# Codificar etiquetas
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_accel_raw)

# División de datos
X_train, X_test, y_train, y_test = train_test_split(
    X_accel_raw, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"✅ Datos preparados:")
print(f"  X_train: {X_train.shape}")  # (n_samples, 250, 6)
print(f"  X_test: {X_test.shape}")
print(f"  Timesteps: {X_train.shape[1]}")
print(f"  Características: {X_train.shape[2]}")

# Crear y entrenar modelo
input_shape = (250, X_train.shape[2])  # (250, 6) para multi-sensor
num_classes = len(label_encoder.classes_)

🚀 Creando ventanas RAW de 250 timesteps...
🔧 Configuración de ventanas RAW:
  Duración: 5s
  Timesteps objetivo: 250
  Frecuencia de muestreo: 20Hz
  Solapamiento: 50%
  Muestras esperadas por ventana: 100
  Paso entre ventanas: 50 muestras
👤 Usuario 1600.0, Actividad A: 3602 muestras
  ✅ Creadas 71 ventanas
👤 Usuario 1600.0, Actividad B: 3604 muestras
  ✅ Creadas 71 ventanas
👤 Usuario 1600.0, Actividad C: 3604 muestras
  ✅ Creadas 71 ventanas
👤 Usuario 1600.0, Actividad D: 3604 muestras
  ✅ Creadas 71 ventanas
👤 Usuario 1600.0, Actividad E: 3604 muestras
  ✅ Creadas 71 ventanas
👤 Usuario 1600.0, Actividad F: 3604 muestras
  ✅ Creadas 71 ventanas
👤 Usuario 1600.0, Actividad G: 3603 muestras
  ✅ Creadas 71 ventanas
👤 Usuario 1600.0, Actividad H: 3603 muestras
  ✅ Creadas 71 ventanas
👤 Usuario 1600.0, Actividad I: 3604 muestras
  ✅ Creadas 71 ventanas
👤 Usuario 1600.0, Actividad J: 3603 muestras
  ✅ Creadas 71 ventanas
👤 Usuario 1600.0, Actividad K: 3604 muestras
  ✅ Creadas 71 ventanas


In [ ]:
model = create_cnn_lstm_model(input_shape=input_shape, num_classes=num_classes)

# Entrenar
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=callbacks
)